In [1]:
# import dependencies
import os
import openai
import pandas as pd

In [3]:
# Retrieve OpenAI API key from os environments
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key[0]

's'

## Tabular Data Setup

In [4]:
# Import previously saved Kaggle sample sales data (https://www.kaggle.com/datasets/kyanyoga/sample-sales-data )
sales_df= pd.read_csv('data/sales_data_sample.csv')

# View imported data
sales_df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


### Create a temporary DB in RAM

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [6]:
# Create sqlite DB
temp_db = create_engine("sqlite:///:memory:", echo=True) # echo will show output

In [7]:
# Push DataFrame to Temp DB
data = sales_df.to_sql(name='Sales',con=temp_db)

2023-03-06 10:31:32,185 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 10:31:32,185 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-06 10:31:32,186 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 10:31:32,187 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-06 10:31:32,188 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-06 10:31:32,189 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-06 10:31:32,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 10:31:32,199 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-03-06 10:31:32,201 INFO sqlalche

In [8]:
# Test db wwith a SQL query
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))

2023-03-06 10:31:33,373 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 10:31:33,374 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-03-06 10:31:33,375 INFO sqlalchemy.engine.Engine [generated in 0.00223s] ()
2023-03-06 10:31:33,377 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
result.all()

[(10032628.85000001,)]

## Take in a Natural Language Request

In [10]:
# The following is how OpenAI suggests the NLP to SQL prompt should look (note that all but last line should start with a #)

# ### Postgres SQL tables, with their properties:       <- tell it what kind SQL DB it is
# #
# # Employee(id, name, department_id)
# # Department(id, name, address)                       <- tell it what tables and columns are in DB
# # Salary_Payments(id, employee_id, amount, date)
# #
# ### A query to list the names of the departments which employed more than 10 employees in the last 3 months                                  <- actual question in natural language
# SELECT                                <- "text completion" so guide it by starting with SELECT

### Define the Table Structure

In [11]:
def define_table_structure(df):
    ts = """ ### sqlite SQL table, with it properties:
    #
    # Sales({})
    #
    """.format(','.join(str(col) for col in df.columns))
    return ts

In [12]:
define_table_structure(sales_df)

' ### sqlite SQL table, with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    #\n    '

### Get Natural Language Query

In [13]:
def human_query():
    hq_text = input("What info do you want?: ")
    return hq_text

In [14]:
human_query()

'SUm of sales by postal code'

### Combine functions to generate a full prompt

In [15]:
def full_prompt(df, human_query_text):
    table_structure = define_table_structure(df)
    query_string = f"### A query to answer: {human_query_text}\nSELECT"
    return table_structure+query_string

In [24]:
hq_text = human_query()

full_prompt(sales_df,hq_text)

' ### sqlite SQL table, with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    #\n    ### A query to answer: return sum of SALES per POSTALCODE\nSELECT'

## Make the call to OpenAI API

In [25]:
response = openai.Completion.create(
    model = 'code-davinci-002',
    prompt = full_prompt(sales_df,hq_text),
    temperature = 0, # we want most probable outcome
    max_tokens = 150,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = ['#',';'] 
    # if you're commenting about code then you've probably already given a satisfactory output
    # also ';' represent the end of a SQL call
)

In [26]:
response

<OpenAIObject text_completion id=cmpl-6r8B5HBiWeMXbb3s0Xbpeon7WWuAr at 0x7f83ecb6f5e0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " POSTALCODE, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY POSTALCODE\nORDER BY TOTAL_SALES DESC\nLIMIT 10"
    }
  ],
  "created": 1678120839,
  "id": "cmpl-6r8B5HBiWeMXbb3s0Xbpeon7WWuAr",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 44,
    "prompt_tokens": 122,
    "total_tokens": 166
  }
}

In [27]:
response['choices'][0]['text']

' POSTALCODE, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY POSTALCODE\nORDER BY TOTAL_SALES DESC\nLIMIT 10'

In [28]:
def handle_response(response):
    sql_query = response['choices'][0]['text']
    if sql_query.startswith(" "):
        sql_query = "SELECT"+sql_query
    else:
        sql_query ="SELECT "+sql_query
    return sql_query

In [35]:
sql_query = handle_response(response)

In [42]:
final_query = ' '.join(sql_query.split('\n'))
final_query

'SELECT POSTALCODE, SUM(SALES) AS TOTAL_SALES FROM Sales GROUP BY POSTALCODE ORDER BY TOTAL_SALES DESC LIMIT 10'

In [43]:
with temp_db.connect() as conn:
    result = conn.execute(text(final_query))

2023-03-06 10:54:54,504 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-06 10:54:54,511 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES) AS TOTAL_SALES FROM Sales GROUP BY POSTALCODE ORDER BY TOTAL_SALES DESC LIMIT 10
2023-03-06 10:54:54,514 INFO sqlalchemy.engine.Engine [generated in 0.01004s] ()
2023-03-06 10:54:54,520 INFO sqlalchemy.engine.Engine ROLLBACK


In [44]:
result.all()

[('28034', 912294.1100000002),
 ('97562', 756752.8500000003),
 ('10022', 560787.7699999998),
 ('94217', 331011.8500000001),
 (None, 272407.14),
 ('50553', 207874.86),
 ('44000', 204304.86),
 ('3004', 200995.40999999997),
 ('79903', 172989.68000000008),
 ('28023', 170257.33000000005)]